In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")

print("Libraries imported successfully!")


In [ ]:
# Create a sample dataset with various data quality issues
data = {
    'customer_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'name': ['John Smith', 'Sarah Johnson', 'Michael Brown', 'Emily Davis', 'Robert Wilson', 
             'Jennifer Lee', 'David Miller', 'Lisa Wang', 'James Taylor', 'Amanda Garcia',
             'Thomas Martin', np.nan, 'Kevin Lewis', 'Michelle Chen', 'Daniel White'],
    'age': [34, -28, 45, 31, 39, 33, 41, 36, 44, 29, 52, 38, 27, 'forty', 35],
    'email': ['john.smith@email.com', 'sarah.johnson@email.com', 'michael.b@email', 'emily.davis@email.com', 
              'robert.wilson@email.com', 'jennifer_lee@email.com', 'david.miller@email.com', 'lisa.wang@email.com', 
              'james.taylor@email.com', 'amanda.garcia@email.com', 'thomas.martin@email.com', 'kevin.lewis@email.com', 
              'michelle.chen@email.com', 'daniel.white@email.com', 'daniel.white@email.com'],
    'purchase_amount': [125.99, 89.50, 250.00, np.nan, 175.25, 0, 99.99, 145.75, 65.50, 199.99, 
                        145.00, 79.99, 299.99, 129.50, 1999.99],
    'purchase_date': ['2023-01-15', '2023-01-16', '2023/01/17', '2023-01-18', '01-19-2023', 
                     '2023-01-20', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24',
                     '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28', '2023-01-29'],
    'category': ['Electronics', 'Clothing', 'Electronics', 'Home', 'Electronics', 
                'Clothing', 'Sports', 'Home', 'Books', 'Electronics',
                'Books', 'Sports', 'Electronics', 'Clothing', 'electronics'],
    'is_member': ['Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 
                 'Y', 'N', 'TRUE', 'FALSE', '1'],
    'discount_code': ['DISC10', 'DISC20', np.nan, 'DISC10', 'DISC20', 'DISC10', np.nan, 'DISC20', 
                     'DISC10', 'DISC20', 'DISC10', 'DISC20', 'DISC10', 'DISC20', 'DISC10']
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the data
print("Sample dataset with data quality issues:")
print(df.head(15))


In [ ]:
# Basic information about the dataset
print("Dataset shape:", df.shape)
print("\nData types:")
print(df.dtypes)

print("\nBasic statistics:")
print(df.describe(include='all'))

print("\nMissing values count:")
print(df.isnull().sum())

print("\nDuplicate rows count:", df.duplicated().sum())


In [ ]:
# Visualize missing values
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap')
plt.tight_layout()
plt.show()

# Handle missing values
df_clean = df.copy()

# 1. Fill missing names with "Unknown"
df_clean['name'] = df_clean['name'].fillna('Unknown')

# 2. Fill missing purchase amounts with the mean
mean_purchase = df_clean['purchase_amount'].mean()
df_clean['purchase_amount'] = df_clean['purchase_amount'].fillna(mean_purchase)

# 3. Fill missing discount codes with "NONE"
df_clean['discount_code'] = df_clean['discount_code'].fillna('NONE')

# Check if missing values were handled
print("Missing values after handling:")
print(df_clean.isnull().sum())


In [ ]:
# Fix the age column
print("Age column before cleaning:")
print(df_clean['age'].head(15))

# Convert age to numeric, errors become NaN
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')

# Replace negative ages with absolute value
df_clean['age'] = df_clean['age'].apply(lambda x: abs(x) if not pd.isna(x) and x < 0 else x)

# Fill NaN values with median age
median_age = df_clean['age'].median()
df_clean['age'] = df_clean['age'].fillna(median_age)

# Convert to integer
df_clean['age'] = df_clean['age'].astype(int)

print("\nAge column after cleaning:")
print(df_clean['age'].head(15))

# Standardize date format
print("\nPurchase date before cleaning:")
print(df_clean['purchase_date'].head(15))

# Function to standardize date format
def standardize_date(date_str):
    try:
        # Try different date formats
        for fmt in ('%Y-%m-%d', '%Y/%m/%d', '%m-%d-%Y'):
            try:
                return pd.to_datetime(date_str, format=fmt).strftime('%Y-%m-%d')
            except:
                continue
        return pd.to_datetime(date_str).strftime('%Y-%m-%d')
    except:
        return None

# Apply the function
df_clean['purchase_date'] = df_clean['purchase_date'].apply(standardize_date)

print("\nPurchase date after cleaning:")
print(df_clean['purchase_date'].head(15))


In [ ]:
# Standardize category names (convert to title case)
print("Categories before standardization:")
print(df_clean['category'].value_counts())

df_clean['category'] = df_clean['category'].str.title()

print("\nCategories after standardization:")
print(df_clean['category'].value_counts())

# Standardize is_member column
print("\nMembership status before standardization:")
print(df_clean['is_member'].value_counts())

# Map various values to True/False
membership_map = {
    'Yes': True, 'Y': True, 'TRUE': True, '1': True, 1: True,
    'No': False, 'N': False, 'FALSE': False, '0': False, 0: False
}

df_clean['is_member'] = df_clean['is_member'].map(membership_map)

print("\nMembership status after standardization:")
print(df_clean['is_member'].value_counts())


In [ ]:
# Check for duplicate customer IDs
print("Duplicate customer IDs:")
print(df_clean[df_clean.duplicated(subset=['customer_id'], keep=False)])

# Check for duplicate emails
print("\nDuplicate emails:")
duplicate_emails = df_clean[df_clean.duplicated(subset=['email'], keep=False)]
print(duplicate_emails)

# Remove duplicate emails (keep first occurrence)
df_clean = df_clean.drop_duplicates(subset=['email'], keep='first')

print("\nDataset shape after removing duplicates:", df_clean.shape)


In [ ]:
# Detect outliers in purchase_amount using box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_clean['purchase_amount'])
plt.title('Purchase Amount Distribution')
plt.tight_layout()
plt.show()

# Calculate IQR for purchase_amount
Q1 = df_clean['purchase_amount'].quantile(0.25)
Q3 = df_clean['purchase_amount'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Outlier boundaries for purchase_amount: Lower = {lower_bound:.2f}, Upper = {upper_bound:.2f}")

# Identify outliers
outliers = df_clean[(df_clean['purchase_amount'] < lower_bound) | 
                    (df_clean['purchase_amount'] > upper_bound)]

print("\nOutliers in purchase_amount:")
print(outliers[['customer_id', 'name', 'purchase_amount']])

# Handle outliers - cap at boundaries
df_clean_no_outliers = df_clean.copy()
df_clean_no_outliers['purchase_amount'] = df_clean_no_outliers['purchase_amount'].clip(lower=lower_bound, upper=upper_bound)

print("\nPurchase amounts after handling outliers:")
print(df_clean_no_outliers[['customer_id', 'name', 'purchase_amount']].head(15))


In [ ]:
# Validate and correct email addresses
def validate_email(email):
    if pd.isna(email):
        return email
    
    # Check if email contains @ and .
    if '@' not in email or '.' not in email.split('@')[1]:
        return None
    
    return email

# Apply validation
df_clean_no_outliers['email_valid'] = df_clean_no_outliers['email'].apply(validate_email)

# Show invalid emails
invalid_emails = df_clean_no_outliers[df_clean_no_outliers['email_valid'].isna()]
print("Invalid email addresses:")
print(invalid_emails[['customer_id', 'name', 'email']])

# Correct specific email (example)
df_clean_no_outliers.loc[df_clean_no_outliers['customer_id'] == 3, 'email'] = 'michael.brown@email.com'
df_clean_no_outliers.loc[df_clean_no_outliers['customer_id'] == 3, 'email_valid'] = 'michael.brown@email.com'

print("\nAfter correction:")
print(df_clean_no_outliers.loc[df_clean_no_outliers['customer_id'] == 3, ['customer_id', 'name', 'email', 'email_valid']])

# Remove the temporary validation column
df_clean_no_outliers = df_clean_no_outliers.drop(columns=['email_valid'])


In [ ]:
# Split name into first_name and last_name
df_clean_no_outliers[['first_name', 'last_name']] = df_clean_no_outliers['name'].str.split(' ', n=1, expand=True)

# Handle cases where last_name might be None
df_clean_no_outliers['last_name'] = df_clean_no_outliers['last_name'].fillna('')

print("Names split into first and last name:")
print(df_clean_no_outliers[['name', 'first_name', 'last_name']].head(15))

# Standardize discount codes (uppercase)
df_clean_no_outliers['discount_code'] = df_clean_no_outliers['discount_code'].str.upper()

print("\nStandardized discount codes:")
print(df_clean_no_outliers['discount_code'].value_counts())


In [ ]:
# Review the final cleaned dataset
print("Original dataset shape:", df.shape)
print("Cleaned dataset shape:", df_clean_no_outliers.shape)

# Final data types
print("\nData types after cleaning:")
print(df_clean_no_outliers.dtypes)

# Basic statistics of cleaned data
print("\nBasic statistics of cleaned data:")
print(df_clean_no_outliers.describe())

# Save the cleaned dataset to CSV
df_clean_no_outliers.to_csv('cleaned_customer_data.csv', index=False)
print("\nCleaned dataset saved to 'cleaned_customer_data.csv'")

# Compare original and cleaned data
print("\nComparison of original vs cleaned data:")
comparison = pd.DataFrame({
    'Original': df.dtypes,
    'Cleaned': df_clean_no_outliers.dtypes
})
print(comparison)
